In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

project_folder = os.getcwd()
for dirname, _, filenames in os.walk(os.path.join(project_folder,'kaggle/input')):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/.DS_Store
/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/order_brush_order.csv


In [128]:
import pandas as pd #imports Python Data Analysis Library

df = pd.read_csv('/Users/haiho/ShopeeCodeLeague/ShopeeCodeLeague2020/1_ORDER-BRUSHING/kaggle/input/order_brush_order.csv')

In [129]:
df.event_time = pd.to_datetime(df.event_time)

# Submission format

In [270]:
# Submission format
import pandas as pd
import numpy as np

submission_shopid = pd.DataFrame(df.shopid.unique())
submission_userid = pd.DataFrame(np.zeros(shape=submission_shopid.shape))
submission = pd.DataFrame(columns = ['shopid', 'userid'])
submission.shopid = df.shopid.unique()
submission.userid = str(0)
submission.to_csv("submission.csv", header=True, index=False)

In [272]:
submission.head()

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,0
3,190969466,0
4,2859407,0


# Preprocessing

In [130]:
df = df.sort_values(by=['event_time'])

In [133]:
df['orderdate'] = [d.date() for d in df['event_time']]
df['ordertime'] = [d.time() for d in df['event_time']]
df['orderhour'] = [d.hour for d in df['ordertime']]

In [136]:
df.head()

,orderid,shopid,userid,event_time,orderdate,ordertime,orderhour
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27,00:00:00,0
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27,00:00:02,0
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27,00:00:05,0


In [287]:
a = df.groupby(['orderdate', 'orderhour', 'shopid'])['userid'].value_counts()
suspicious_shops = {}
for index, value in a.iteritems():
    if value > 3:
        suspicious_shopid = index[2]
        suspicious_userid = index[3]
        if suspicious_shopid not in suspicious_shops.keys():
            suspicious_shops[suspicious_shopid]=str(suspicious_userid)
        else:
            suspicious_shops[suspicious_shopid]+="&"+str(suspicious_userid)
print(suspicious_shops)

{147941492: '147478511&163220673&199110270&208806468&58009118&2812894&77645501&78274359&212480041&164098479&50499134&190222721&43652561&4769572&211429582&205951406&213906397&89835507&59282182&189029587&71026438&88732127&86875149&208806468&37434703&50499134&197551102&213526271&52097335&5333888&95619116&7616036&16950820&86051997&213787697&22184&164098479&205583383&86875149&166081737&208806468&52255254&80060796&13487830&205951406&7616036&154598501&52097335&205583383&198218978&3142265&203538523&3142265&9613337&210362283&195350861&2284151&208806468&9613337&3142265&19970394&206794371&88587531&197551102&23857471&52097335&84226482&12282583&58009118&45163392&199110270&190222721&113418397&43652561&3142265&3524519&55372731&1997501&9535411&213162186&203742758&206585991&50499134&101092594&103256866&3142265&48793634&113830512&164098479&205046026&186403294&205435395&107331095&207802408&29815158&104066552&19443866&23961096&52415138&80060796&3766486&208806468&7872248&30279967&55446317&112716544&5225525

In [288]:
for index, value in submission.iterrows():
    if value[0] in suspicious_shops.keys():
        submission.iloc[index,1] = suspicious_shops[value[0]]

In [289]:
submission.head(100)

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,9753706
3,190969466,0
4,2859407,0
...,...,...
95,33380999,0
96,68973461,0
97,163522251,0
98,32504026,10221040


In [290]:
submission.to_csv("submission.csv", header=True, index=False)